In [7]:
import pandas as pd
from tqdm import tqdm
import pandas as pd
from chromadb import PersistentClient
from chromadb.utils import embedding_functions
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer


In [3]:
df = pd.read_csv('DataSets\clean_eng_subtitles.csv')

In [4]:
df=df.drop('Unnamed: 0',axis=1)
df['file_content'] = df['file_content'].str.lower()
df['name'] = df['name'].str.lower()
df['name'] = df['name'].str.replace('eng 1cd', '').str.strip()

In [12]:
model = SentenceTransformer('bert-base-nli-mean-tokens')
embeddings = model.encode(df['file_content'])

In [28]:
embedding_dict = {}
for i, embedding in enumerate(embeddings):
    embedding_dict[i] = embedding

for i in range(1):
    print(f"Embedding {i}: {embedding_dict[i]}")

Embedding 0: [-6.46385193e-01  8.08023810e-01  8.63844216e-01  1.48276269e-01
 -1.38662517e-01 -1.08571601e+00  1.01714432e+00  6.78809453e-03
  2.37831891e-01 -1.21645987e-01 -2.64770687e-01  3.78210336e-01
  8.22313666e-01  1.60806552e-01 -1.68955612e+00  7.80357838e-01
  1.94873184e-01  1.06023448e-02 -1.67095698e-02 -2.06807956e-01
 -1.94412153e-02 -8.84590924e-01  1.79695234e-01 -1.15603572e-02
  7.76240647e-01  8.87314320e-01  2.43718714e-01  7.45085299e-01
 -6.29639030e-01  2.56271571e-01 -2.61619270e-01  9.53797579e-01
 -2.51813233e-02 -1.83548540e-01  2.76170969e-01  1.12556815e+00
  3.33631545e-01  7.31134057e-01 -1.58313528e-01  8.61065462e-02
 -1.01286829e-01  3.61391678e-02 -3.80047500e-01 -5.98785937e-01
 -7.07759857e-01 -2.70011663e-01  3.59028727e-01  6.54030442e-01
  9.48156178e-01 -1.13418365e+00  5.47300458e-01  3.09271097e-01
  1.55863714e+00 -3.37997973e-01 -2.35103771e-01 -1.06949076e-01
  1.76267296e-01 -4.94238257e-01 -7.65732765e-01 -4.98540014e-01
 -1.09627712

In [66]:
def search(query, data, embeddings, model):
    
    query_embedding = model.encode([query])[0]
    similarities = cosine_similarity([query_embedding], embeddings)
    
    top_n = 10
    top_indices = np.argsort(similarities[0])[-top_n:][::-1]  
    results = [(data['name'][i], similarities[0][i]) for i in top_indices]
    
    return results

In [67]:
query = input("Enter your search query of English movies and series: ")
search_results = search(query, df, embeddings, model)
for result in search_results:
    print("Document:", result[0])
    print("Similarity Score:", result[1])
    print()

Enter your search query of English movies and series:  drama


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Document: the imperfects s01 e10 destroy all monsters (2022)
Similarity Score: 0.66330755

Document: alice in borderland s02 e07 episode 2 7 (2022)
Similarity Score: 0.59948903

Document: tooth pari when love bites s01 e08 episode 1 8 (2023)
Similarity Score: 0.5738974

Document: il grande gioco s01 e04 contropiede (2022)
Similarity Score: 0.54872626

Document: the recruit s01 e01 i n a s i a l (2022)
Similarity Score: 0.47358322

Document: bleach s07 e19 oath back here alive again (2007)
Similarity Score: 0.46810183

Document: bleach s08 e16 the moment of conclusion the end of grimmjow (2008)
Similarity Score: 0.46810183

Document: bleach s07 e20 the raging storm encounter with the dancing arrancar (2007)
Similarity Score: 0.46810183

Document: bleach s08 e05 ishida pesche the united attack of friendship (2008)
Similarity Score: 0.46810183

Document: bleach s08 e13 ishidas strategy the 20second offense and defense (2008)
Similarity Score: 0.46810183



In [22]:
# Loading and Preprocessing
data = pd.read_csv(r"/kaggle/input/eng-subtitle/clean_eng_subtitles_csv.csv")
data = data.drop('Unnamed: 0',axis=1)
data['file_content'] = data['file_content'].str.lower()
data['name'] = data['name'].str.lower()
data['name'] = data['name'].str.replace('eng 1cd', '').str.strip()

# Initialize lists for documents, metadatas, and ids
documents = data['file_content'].tolist()  # Store file content as documents
metadatas = data[['num', 'name']].to_dict(orient='records')  # Store num and name as metadata
ids = data['num'].astype(str).tolist()  # Convert num to string and store as IDs


In [ ]:
chroma_client = PersistentClient(path="my_vectordb")

model_name = "all-mpnet-base-v2"
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name=model_name
)

In [ ]:
collection_name = "eng_subtitles_collection"
try:
    collection = chroma_client.get_collection(name=collection_name, embedding_function=sentence_transformer_ef)
except ValueError:
    print(f"Collection '{collection_name}' does not exist. Creating a new collection.")
    collection = chroma_client.create_collection(name=collection_name, embedding_function=sentence_transformer_ef)

In [18]:
print("Adding documents to the collection...")
with tqdm(total=len(documents), desc="Progress", unit="document", ncols=100) as pbar:
    # Add documents, metadata, IDs, and embeddings in a single operation
    collection.add(
        documents=documents,
        embeddings=embeddings,
        metadatas=metadatas,
        ids=ids
    )
    pbar.update(len(documents))

print("Finished adding documents to the collection.")

Adding documents to the collection...


Progress: 100%|████████████████████████████████████████| 24749/24749 [03:07<00:00, 132.30document/s]

Finished adding documents to the collection.


In [24]:

def search(query, collection):
    query_embedding = model.encode([query])
    results = collection.query(
        query_embeddings=query_embedding,
        n_results=10,  # Specify the number of results you want to retrieve
        include=['documents', 'distances', 'metadatas']  # Remove 'ids'
    )

    for j in range(len(results['ids'][0])):
        movie_id = results["ids"][0][j]
        distance = results['distances'][0][j]
        metadata = results['metadatas'][0][j]
        
        print(f"Movie ID (num): {movie_id}")
        print(f"Similarity Score (distance): {distance:.3f}")
        print(f"Movie Name: {metadata['name']}")

In [25]:
query_text = input("Please enter your search query: ")
search(query_text, collection)

Please enter your search query:  action


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Movie ID (num): 9233588
Similarity Score (distance): 174.313
Movie Name: the imperfects s01 e10 destroy all monsters (2022)

Movie ID (num): 9306106
Similarity Score (distance): 240.475
Movie Name: warrior nun s02 e08 jeremiah 2913 (2022)

Movie ID (num): 9489586
Similarity Score (distance): 298.971
Movie Name: batwheels s01 e17 holidays on ice (2022) eng 1cd

Movie ID (num): 9376827
Similarity Score (distance): 300.365
Movie Name: bang s01 e04 episode 1 4 (2017) eng 1cd

Movie ID (num): 9206754
Similarity Score (distance): 300.687
Movie Name: its always sunny in philadelphia s10 e01 the gang beats boggs (2015) eng 1cd

Movie ID (num): 9229488
Similarity Score (distance): 300.813
Movie Name: rush s02 e13 episode 2 13 (2009) eng 1cd

Movie ID (num): 9397061
Similarity Score (distance): 300.844
Movie Name: v s01 e01 liberation day (1984) eng 1cd

Movie ID (num): 9233848
Similarity Score (distance): 300.884
Movie Name: wedding season s01 e02 episode 1 2 (2022) eng 1cd

Movie ID (num): 927